In [11]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as xeet
from glob import glob
import os

In [3]:
path = glob('./images/*.xml')
dictnory_labels = dict(file_path=[],xmin=[],xmax=[],ymin=[],ymax=[],filename=[],width=[],height=[])
for file_name in path:
    #getting information from the xml file
    information = xeet.parse(file_name)
    root_val = information.getroot()
    object_mem = root_val.find('object')
    label_information = object_mem.find("bndbox")
    xmax = int(label_information.find('xmax').text)
    xmin = int(label_information.find('xmin').text)
    ymax = int(label_information.find('ymax').text)
    ymin = int(label_information.find('ymin').text)
    
    name = root_val.find('filename').text
    filename = f'./images/{name}'

    # width and height
    parser_size = root_val.find('size')
    width = int(parser_size.find('width').text)
    height = int(parser_size.find('height').text)
    #appedning filenames and xmax, xmin, ymin, ymax to single dict

    dictnory_labels['file_path'].append(file_name)
    dictnory_labels['xmin'].append(xmin)
    dictnory_labels['xmax'].append(xmax)
    dictnory_labels['ymin'].append(ymin)
    dictnory_labels['ymax'].append(ymax)
    dictnory_labels['filename'].append(filename)
    dictnory_labels['width'].append(width)
    dictnory_labels['height'].append(height)

In [4]:
df =pd.DataFrame(dictnory_labels)
df.to_csv("car_labels.csv",index= False)

In [5]:
df

,file_path,xmin,xmax,ymin,ymax,filename,width,height
0,./images\N0.xml,164,274,224,248,./images/N0.jpeg,442,333
1,./images\N1.xml,1087,1407,647,730,./images/N1.jpeg,1920,1080
2,./images\N100.xml,129,307,302,355,./images/N100.jpeg,450,417
3,./images\N101.xml,21,140,124,164,./images/N101.jpeg,249,239
4,./images\N102.xml,160,320,203,253,./images/N102.jpeg,478,395
...,...,...,...,...,...,...,...,...
224,./images\N95.xml,11,420,162,397,./images/N95.jpeg,450,600
225,./images\N96.xml,116,382,132,205,./images/N96.jpeg,450,290
226,./images\N97.xml,171,297,223,259,./images/N97.jpeg,446,282
227,./images\N98.xml,563,678,205,241,./images/N98.jpeg,704,330


In [6]:
#for yolo model we consider center_x,center_y,width,height. here we have width and height, so we calcualte center_x, center_y

df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [7]:
df.head()

,file_path,xmin,xmax,ymin,ymax,filename,width,height,center_x,center_y,bb_width,bb_height
0,./images\N0.xml,164,274,224,248,./images/N0.jpeg,442,333,0.495475,0.708709,0.248869,0.072072
1,./images\N1.xml,1087,1407,647,730,./images/N1.jpeg,1920,1080,0.649479,0.637500,0.166667,0.076852
2,./images\N100.xml,129,307,302,355,./images/N100.jpeg,450,417,0.484444,0.787770,0.395556,0.127098
3,./images\N101.xml,21,140,124,164,./images/N101.jpeg,249,239,0.323293,0.602510,0.477912,0.167364
4,./images\N102.xml,160,320,203,253,./images/N102.jpeg,478,395,0.502092,0.577215,0.334728,0.126582


In [8]:
df.shape

(229, 12)

In [9]:
#spliting the data into testing and training
df_train = df.iloc[:195]
df_test = df.iloc[195:]

In [12]:
#Getting data from csv and adding new te
from shutil import copy

train_folder = './data_im/train'

values = df_train[['filename','center_x','center_y','bb_width','bb_height']].values
for filename, b_center_x,b_center_y, b_width, b_height in values:
    image_name = os.path.split(filename)[-1]
    txt_file_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(train_folder,image_name)
    dst_label_file = os.path.join(train_folder,txt_file_name+'.txt')
    
    # copy each image into the folder
    copy(filename,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {b_center_x} {b_center_y} {b_width} {b_height}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()


In [13]:
from shutil import copy

test_folder = './data_im/test'

values = df_test[['filename','center_x','center_y','bb_width','bb_height']].values
for filename, b_center_x,b_center_y, b_width, b_height in values:
    image_name = os.path.split(filename)[-1]
    txt_file_name = os.path.splitext(image_name)[0]
    
    dst_image_path = os.path.join(test_folder,image_name)
    dst_label_file = os.path.join(test_folder,txt_file_name+'.txt')
    
    # copy each image into the folder
    copy(filename,dst_image_path)

    # generate .txt which has label info
    label_txt = f'0 {b_center_x} {b_center_y} {b_width} {b_height}'
    with open(dst_label_file,mode='w') as f:
        f.write(label_txt)
        
        f.close()
